# Server connection check

In [66]:
from typing import Any

from twisted.mail.smtp import SUCCESS

# In this notebook we have trained the pytorch models for the adversarial and XAI data of the FGSM attack conducted on the DeepMIMO dataset. This notebook only contains the Pytorch based neural network.

print('hello')
import os
from sys import executable as exec
print('current working directory: ', os.getcwd())
print('current conda environment: ', exec)

#--notebook-dir=/home/spatial01/RND/adversarial-detector/adversarial-detector-xai

hello
current working directory:  /home/spatial01/RND/adversarial-detector/adversarial-detector-xai
current conda environment:  /home/spatial01/anaconda3/envs/netslab_lime_defense/bin/python


In [67]:
# %load_ext autoreload
# %autoreload 2

## Testing if Torch is working correclty
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.2909, 0.4198, 0.4386],
        [0.6161, 0.0260, 0.7284],
        [0.9474, 0.8516, 0.4202],
        [0.6345, 0.9185, 0.9822],
        [0.2324, 0.7525, 0.2175]])


# Imports

In [68]:

# Eager execution shit in tensorflow

import os
import glob as glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import datetime as dt
import pandas as pd
import deprecation
from pprint import pp as pprint
from tqdm import tqdm

# %% Machine Learning
from sklearn.model_selection import train_test_split

# reload and load custom modules
import importlib
import myutils
import codb_utils
importlib.reload(codb_utils)
importlib.reload(myutils)
from myutils import create_dir_if_not_exist, get_latest_file, to_dataLoader, PyTorchPredWrapper, load_latest_model, get_latest_model, conv_double_to_single_col_binary_labels

# Experiment loop
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_recall_fscore_support

# pytorch
from torch import optim, nn, Tensor
import torch
import lightning as L
from torchmetrics import Accuracy

plt.rcParams['figure.figsize'] = [12, 8] # Set default plot size

# paths
# Change the date here to use the files from a different date
proj_root = os.getcwd()
# Loading datasets
load_dlcb_data_dir = os.path.join(os.getcwd(), "data/DeepMIMO_data/DLCB_dataset_21-07-24/")
load_adv_fgsm_data_dir = os.path.join(proj_root, "results/Novel_DMIMO_21-03-24/AdvData/")
load_adv_cw_data_dir = os.path.join(proj_root, "results/Novel_DMIMO_28-04-24/AdvData/")
load_all_adv_data_dir = os.path.join(proj_root, "results/Novel_DMIMO_02-05-24/AdvData/")
load_xai_data_dir = os.path.join(proj_root, "results/Novel_DMIMO_12-04-24/Explanations/")
# Loading logs
load_lighting_logs_dir = os.path.join(proj_root, "lightning_logs/")
# Loading results and observations
load_results_dir = os.path.join(proj_root, "results/Novel_DMIMO_15-04-24/Observations/")
# Loading trained models
load_adv_detector_dir = os.path.join(proj_root, "results/Novel_DMIMO_11-04-24/Models/adv_detector/")
load_dlcb_models_dir = os.path.join(proj_root, "results/Novel_DMIMO_27-01-25 - EBY/Models/dlcb_models/")
# EBY data directories
load_eby_data_dir_v1 = os.path.join(os.getcwd(), "data/EBY_data/V1/")

path_base=os.path.join(os.getcwd(), "results/Novel_DMIMO_")
# Create directories
results_dir = create_dir_if_not_exist(os.path.join(os.getcwd(), "results/Novel_DMIMO_"+dt.datetime.now().strftime("%d-%m-%y") + " - EBY"))
results_figs_dir = create_dir_if_not_exist(os.path.join(results_dir, 'Figs'))
results_data_dir = create_dir_if_not_exist(os.path.join(results_dir, 'Observations'))
results_adv_data_dir = create_dir_if_not_exist(os.path.join(results_dir, 'AdvData'))
results_explanations_dir = create_dir_if_not_exist(os.path.join(results_dir, 'Explanations'))
results_models_dir = create_dir_if_not_exist(os.path.join(results_dir, 'Models'))
results_xai_detector_dir = create_dir_if_not_exist(os.path.join(results_models_dir, 'xai_detector'))
results_adv_detector_dir = create_dir_if_not_exist(os.path.join(results_models_dir, 'adv_detector'))
results_dlcb_models_dir = create_dir_if_not_exist(os.path.join(results_models_dir, 'dlcb_models'))

# CONFIGURATION
# Make it False to train the model
LOAD_XAI_ADVERSARIAL_MODEL_FROM_FILE = False
modifier_txt_direct_adv_det = 'Full_direct_adv_DF, lr 1e-3, three_layers'
modifier_txt_xai_adv_det = 'Full_XAI_DF, lr 1e-3'
modifier_txt_xgboost_zero_day='Unseen_XGBoost_Tr-FGSM_Ts-FGSM-CWL2'
modifier_txt_skl_mlp_zero_day='Unseen_SKL_MLP_Tr-FGSM_Ts-FGSM-CWL2'
modifier_txt_skl_mlp_adv_det = 'SKL_MLP_Tr-FGSM_Ts-FGSM'
modifier_txt_xgboost_adv_det = 'XGBoost_Tr-FGSM_Ts-FGSM'
# Make it False to train the model
LOAD_DIRECT_ADVERSARIAL_MODEL_FROM_FILE = False
TRAIN_XGBOOST_ADV_ZERO_DAY=True
TRAIN_SKL_MLP_ADV_ZERO_DAY=True
# Make this 2 or 3 for testing and 1.0 (float) for the full dataset
CONFIG_TRAIN_LIMIT = 1.0
# Batch size for both direct and XAI detectors
DMAD_BATCH_SIZE = 32


data_directories = {
    'dlcb_data_load': load_dlcb_data_dir,
    'dlcb_model_save': results_dlcb_models_dir,
    'dlcb_model_load': load_dlcb_models_dir,
    # 'dlcb_model_load': results_dlcb_models_dir, # this line loads the models from the currently saving folder. To use a fixed folder, use the above commented line
    'fgsm_data_load': load_adv_fgsm_data_dir,
    'xai_data_load': load_xai_data_dir,
    'adv_data_save': results_adv_data_dir,
    'xai_data_save': results_explanations_dir,
    'eby_data_load': load_eby_data_dir_v1
}

import tensorflow as tf

try:
    importlib.reload(tf)
except ValueError as ve:
    print("Eager execution is already enabled: ", ve)

MODEL_TRAINING = False
if MODEL_TRAINING:
    LOAD_MODEL=False
    # tf.compat.v1.disable_eager_execution()
    tf.compat.v1.enable_eager_execution()
elif not MODEL_TRAINING:
    LOAD_MODEL=True
    tf.compat.v1.disable_eager_execution()
    CREATE_ATTACK_DATASETS = LOAD_MODEL
# Change this when you want to train the DLCB original tensorflow model
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass


# Stop running the main loop
RUN_LOOP = False
GENERATE_EXPL_PLOTS = False
TDD_loop = False
RUNTIME_VIZ = False

######### Warning
## It is pertinent that you use the following versions of the packages
# tensorflow==2.14.0
# keras==2.14.0
# tensorflow-addons>=0.13.0
# art==1.17.0
# Otherwise it might give trouble with placeholder and tensorflow v2 related issues.
# The IDS model used must have the output shape with Dense(2)
# art generator takes numpy arrays only for generation. No pandas dataframes to generate_adversarial_data method.

Eager execution is already enabled:  tf.enable_eager_execution must be called at program startup.


# Adversarial data creation

In [69]:
from art.estimators.classification import KerasClassifier
from codb_utils import train_codb_model, codb_train_test_split
from myutils import evasion_attacks, NotFittedError, generate_art_based_evasion_data
from pprint import pp as pprint

from art.attacks.evasion import FastGradientMethod, ZooAttack, CarliniL2Method
from art.attacks import EvasionAttack

In [70]:
# Load the data
def load_all_data(folder_path):
    # Dictionary to store DataFrames
    dataframes_dict = {}
    dos_label_map = {}

    # Iterate through files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file is a CSV
        if file_name.endswith('.csv'):
            # Read the CSV file into a DataFrame
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)

            # Store the DataFrame in the dictionary with the filename as the key
            dict_key = file_name.split(".")[0]
            dict_key = dict_key.split("_")[-1]
            if dict_key.endswith("N"):
                dataframes_dict[dict_key] = df, pd.DataFrame(np.zeros((df.shape[0], 1), dtype=int))
                dos_label_map[0] = "normal"
            elif dict_key.endswith("tcp"):
                dataframes_dict[dict_key] = df, pd.DataFrame(np.ones((df.shape[0], 1), dtype=int))
                dos_label_map[1] = "tcp"
            elif dict_key.endswith("dns"):
                dataframes_dict[dict_key] = df, pd.DataFrame(np.ones((df.shape[0], 1), dtype=int)*2)
                dos_label_map[2] = "dns"
            elif dict_key.endswith("ntp"):
                dataframes_dict[dict_key] = df, pd.DataFrame(np.ones((df.shape[0], 1), dtype=int)*3)
                dos_label_map[3] = "ntp"

    return dataframes_dict, dos_label_map

# Prep the data
def get_X_y_from_data_dict(data_dict):
    dfs = []
    ys = []
    for _, (df, y) in data_dict.items():
        dfs.append(df)
        ys.append(y)
    X = pd.concat(dfs, ignore_index=True)
    y = pd.concat(ys, ignore_index=True)
    return X, y

def binarize_labels_for_ids(y):
    # convert multi label y to binary labels
    y_bin = np.zeros((y.shape[0], 1), dtype=int)
    y_bin[y > 0] = 1
    y_bin_2 = np.zeros((y_bin.shape[0], 2), dtype=int)
    # expand y_bin to two columns with one hot encoding
    for i, yb in enumerate(y_bin):
        y_bin_2[i, yb] = 1
    y_index = y
    return y_bin_2, y_index, {0: "normal", 1: "attack"}

def save_per_attk_adv_data(folder_path, modifier, X, y=None):
    np.save(os.path.join(folder_path, f"adv_X_{modifier}_{dt.datetime.now().strftime('%H-%M-%S')}.npy"), X, allow_pickle=False)
    if y is not None:
        np.save(os.path.join(folder_path, f"adv_y_{modifier}_{dt.datetime.now().strftime('%H-%M-%S')}.npy"), y, allow_pickle=False)
    return True

In [71]:
import keras
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

def train_encoder_decoder_for_adv_gen(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2025)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=2025)

    # Scale the data
    # scaler = StandardScaler()
    # X_train = scaler.fit_transform(X_train)
    # X_test = scaler.transform(X_test)
    input_shape = (X_train.shape[1],)
    # Define the encoder-decoder model
    # Encoder
    model = keras.Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))

    # Decoder / Classifier
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss=BinaryCrossentropy(),
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train,
                        epochs=5,
                        batch_size=32,
                        validation_data=(X_val, y_val),
                        verbose=1,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])

    # save the model
    model_name = "Ericsson-IDS"
    file_path=os.path.join(data_directories['dlcb_model_save'], f"{model_name}_{dt.datetime.now().strftime('%H-%M-%S')}.keras")
    model.save(file_path)

    split_data = [X_train, X_test, y_train, y_test]
    # Evaluate the model
    return model, split_data

In [72]:
def generate_adversarial_data(ids_model, attack_param_dict, input_samples):
    """
    Generates adversarial data for the given attack types and parameters

    Parameters
    ----------
    attack_param_dict: A dictionary containing the attack types and their parameters. Here the FastGradientMethod is of type art.attack.EvasionAttack type
        attack_param_dict={
                            'FastGradientMethod_0.2': (FastGradientMethod, {'eps':0.2}),
                            'FastGradientMethod_0.3': (FastGradientMethod, {'eps':0.3})
                        }
    dataset: Default would be the test dataset in the dlcb_sub_datasets

    Returns
    -------
    total_adv_data_dict: A dictionary containing the adversarial data for each model and each label in the attack params dict
    """
    print(f"Generating adversarial data for model for IDS model")
    adv_wrapped_model = KerasClassifier(model=ids_model, use_logits=False)
    adv_data_dict = {}
    for attack_label, (attack_method, params) in tqdm(attack_param_dict.items()):
        try:
            print(f"Generating {attack_label} attack dataset")
            adv_data_dict[attack_label] = generate_art_based_evasion_data(wrapped_model=adv_wrapped_model, attack_method=attack_method, X=input_samples, **params)
        except Exception as e:
            print(f"Could not generate the attack dataset for {attack_label} due to {e}")
            continue
        # save right after the generation
        save_data_directory = data_directories['adv_data_save']
        model_name = f"Ericsson-IDS"
        save_per_attk_adv_data(save_data_directory, f"{model_name}_{attack_label}", adv_data_dict[attack_label])
        print(f"[SUCCESS] Adversarial data for {attack_label} attack saved for Ericsson-IDS model")
    return adv_data_dict

# write to save all the data adv and original and the model in one .npy file for the run session.
def save_all_together(adv_data_dict, X, y_index, save_data_directory=None, **kwargs):
    model_name = kwargs['model_name']; kwargs.pop('model_name', None) # remove key
    if save_data_directory is None:
        save_data_directory = data_directories['adv_data_save']
    save_dict = {}
    save_dict['adv_data_dict'] = adv_data_dict
    save_dict['X'] = X
    save_dict['y_index'] = y_index
    np.save(os.path.join(save_data_directory, f"AdvPack_{model_name}_{kwargs}_{dt.datetime.now().strftime('%H-%M-%S')}.npy"), X, allow_pickle=False)
    print("[SUCCESS] All the data is packed and saved successfully!")
    return True

In [73]:
# Specify the folder path
folder_path = load_eby_data_dir_v1

# Read CSV files and store them in a dictionary
eby_all_data = load_all_data(load_eby_data_dir_v1)

# Visualize the read data
print("All data sizes: ", eby_all_data[0]["N"][1].shape)
print("All data label_map: ", eby_all_data[1])

# binarise the data for IDS
X, y = get_X_y_from_data_dict(eby_all_data[0])
y, y_index, bin_label_map = binarize_labels_for_ids(y)
print(X.shape, y.shape)
print(y.sum(axis=0))
# Train the ids model
if MODEL_TRAINING:
    ids_model, split_data = train_encoder_decoder_for_adv_gen(X, y)
    # Evaluate the IDS mode;
    test_loss, test_accuracy = ids_model.evaluate(split_data[1], split_data[3], verbose=0)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
elif LOAD_MODEL:
    load_path_extension="Ericsson-IDS*.keras"
    print(f"Loading model {data_directories['dlcb_model_load']}")
    try:
        latest_filename= get_latest_file(data_directories['dlcb_model_load'], load_path_extension)
        ids_model = keras.models.load_model(latest_filename)
        print(f"Model Successfully Loaded from {latest_filename}")
    except Exception as e:
        print(f"Model loading Failed from {latest_filename}")
        print(e)

All data sizes:  (8999, 1)
All data label_map:  {2: 'dns', 1: 'tcp', 0: 'normal', 3: 'ntp'}
(18965, 127) (18965, 2)
[8999 9966]
Loading model /home/spatial01/RND/adversarial-detector/adversarial-detector-xai/results/Novel_DMIMO_27-01-25 - EBY/Models/dlcb_models/
Model Successfully Loaded from /home/spatial01/RND/adversarial-detector/adversarial-detector-xai/results/Novel_DMIMO_27-01-25 - EBY/Models/dlcb_models/Ericsson-IDS_09-14-57.keras


In [75]:
import art
art.__version__

'1.17.0'

In [78]:
# Attack data generation

from art.attacks.evasion import FastGradientMethod, ZooAttack, CarliniL2Method, DeepFool, BasicIterativeMethod, ProjectedGradientDescent, SaliencyMapMethod, TargetedUniversalPerturbation, UniversalPerturbation, Wasserstein, CarliniLInfMethod

if CREATE_ATTACK_DATASETS:
    # attacks_and_params={
    #     'FastGradientMethod_0.2': (FastGradientMethod, {'eps':0.2})
    #     , '': (FastGradientMethod, {'eps':0.2})
    #     #, 'CarliniL2Method_2': (CarliniL2Method, {'max_iter':2, 'learning_rate':0.1})
    # }
    eps_val = 0.9
    adv_datasize="Full"
    attacks_and_params={
        'DeepFool_max_iter_10': (DeepFool, {'max_iter':10, 'batch_size':32, 'nb_grads':1, 'gen_kwargs': {'y': y}})
        ,'CarliniLInfMethod': (CarliniLInfMethod, {'max_iter':2, 'learning_rate':0.1})
        ,'UniversalPerturbation': (UniversalPerturbation, {'attacker':'fgsm', 'gen_kwargs': {'y':y}})
        ,'TargetedUniversalPerturbation': (TargetedUniversalPerturbation, {'gen_kwargs': {'y':y}})
        # ,'SaliencyMapMethod': (SaliencyMapMethod, {})
        ,'ProjectedGradientDescent': (ProjectedGradientDescent, {'eps':eps_val})
        ,'BasicIterativeMethod_0.2': (BasicIterativeMethod, {'eps':eps_val})    # The Basic Iterative Method is the iterative version of FGM and FGSM.
        ,'FastGradientMethod_0.2': (FastGradientMethod, {'eps':eps_val})
        ,'CarliniL2Method_0.2': (CarliniL2Method, {'confidence':eps_val})
    }
    adv_data_dict = generate_adversarial_data(ids_model, attacks_and_params, X.values)
    save_all_together(adv_data_dict, X, y_index, save_data_directory=None, eps=eps_val, model_name="Ericsson-IDS", data_size=adv_datasize)

    exit()
# for the full dataset use concatenate_sub_datasets('X') as the input for generate_adversarial_data function

Generating adversarial data for model for IDS model


  0%|          | 0/8 [00:00<?, ?it/s]

Generating DeepFool_max_iter_10 attack dataset


DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

 12%|█▎        | 1/8 [00:01<00:08,  1.26s/it]

...

 75%|███████▌  | 6/8 [00:04<00:01,  1.85it/s]

[SUCCESS] Adversarial data for BasicIterativeMethod_0.2 attack saved for Ericsson-IDS model
Generating FastGradientMethod_0.2 attack dataset
[SUCCESS] Adversarial data for FastGradientMethod_0.2 attack saved for Ericsson-IDS model
Generating CarliniL2Method_0.2 attack dataset


C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:06<00:00,  1.18it/s]

[SUCCESS] Adversarial data for CarliniL2Method_0.2 attack saved for Ericsson-IDS model
[SUCCESS] All the data is packed and saved successfully!


In [10]:
# Document all the attack dataset types by reading the art documentation under evasion attacks
# LowProFool is a specific attack for tabular data ... Try to implement it for the DLCB dataset
# ThresholdAttack - This attack requires estimator clip values to be defined. try to solve this
# Adversarial Robustness Assessment: Why both L0 and L∞ Attacks Are Necessary (https://arxiv.org/abs/1906.06026)
# ShadowAttack - Breaking certified defenses: Semantic adversarial examples with spoofed robustness certificates
# TargetedUniversalPerturbation - Calculates a universal adversarial perturbation (UAP) but uses fgsm at the backend (can change) https://arxiv.org/abs/1911.06502
# UniversalPerturbation - Calculates a universal adversarial perturbation (UAP) but uses fgsm at the backend (can change) https://arxiv.org/abs/1610.08401
# Wasserstein seem to run into an overflow error in _conjugate_sinkhorn for exponential beta when generating data.
# JSMA SaliencyMapMethod took a long time to run even one datapoint. Stopped after one data point.
# UniversalPerturbation shows like it creates only a small dataset in the logs. TargetedUniversalPerturbation weren't even shown on logs

# Attack data loading

In [12]:
# create separate arrays for attack data testing

# loading all the datasets

# show all the files in results/*/AdvData

def load_all_adv_data(data_dir):
    return {f.split('/')[-1].split('_')[5]: np.load(f) for f in glob.glob(os.path.join(data_dir, 'adv_*.npy'))}

adv_data = load_all_adv_data(load_all_adv_data_dir)
unseen_attack = {'CarliniL2Method': adv_data.pop('CarliniL2Method')}

In [13]:
#concatenate the adversarial data
def concatenate_adv_data(adv_data_dict):
    adv_data_ls = []
    labels_ls = []
    label_map = {}
    for i, (key, value) in enumerate(adv_data_dict.items()):
        adv_data_ls.append(value)
        labels_ls.append(np.ones((value.shape[0],1), dtype=int)*int(i+1))
        label_map[i+1] = key
    X_full = np.concatenate(adv_data_ls, axis=0)
    y_full = np.concatenate(labels_ls, axis=0) 
    return X_full, y_full, label_map

full_adv_X, full_adv_y, label_map = concatenate_adv_data(adv_data)
np.unique(full_adv_y, return_counts=True)

(array([1, 2, 3, 4, 5, 6, 7]),
 array([10897, 10897, 10897, 10897, 10897, 10897, 10897]))

In [14]:
dmaddata_xai = DmadDatasets(data_directories)
adv_det_datasets = dmaddata_xai.adv_detector_task(test_size=0.2, validation_size=None, verbose=True,
                                                  multidim_labels=False)

/tmp/ipykernel_3484888/3799587416.py:249: DeprecatedWarning: _create_detector_labels is deprecated as of 0.1.0. Use the create_detector_full_ds_adding_labels for multiple adversarial and benign dataset
  det_In, det_out = DmadDatasets._create_detector_labels(self.adv_X, self.dlcb_X)


X_train: (87169, 256)
X_test: (21793, 256)
y_train: (87169,)
y_test: (21793,)


In [15]:
benign_x = adv_det_datasets['X_train'][:full_adv_X.shape[0]]
benign_y = np.zeros((benign_x.shape[0],1), dtype=int)
label_map[0] = 'Benign'
X_full = np.concatenate([benign_x, full_adv_X], axis=0)
y_full = np.concatenate([benign_y, full_adv_y], axis=0)

In [16]:
np.unique(y_full, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([76279, 10897, 10897, 10897, 10897, 10897, 10897, 10897]))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=42, stratify=y_full)
